In [10]:
import numpy as np
import sys, os
import matplotlib.pyplot as plt

# Automatically detect project root (folder containing `lib`)
current_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(project_root)

print("Added to path:", project_root)


from lib import Model, Dense, Tanh, Sigmoid, MSELoss, SGD

Added to path: d:\omar\Summer 2025\CI\project\Neural-Network-Library_Advanced-Applications


In [11]:
def gradient_check(model, X, y, epsilon=1e-5, tolerance=1e-6):
    """
    Numerical gradient checking using finite differences.
    """

    # Analytical gradients
    preds = model._forward(X)
    model._loss.forward(preds, y)
    grad = model._loss.backward()
    model._backward(grad)

    print("Running gradient check...")

    for layer in model.layers:
        if not hasattr(layer, "parameters"):
            continue

        for param, grad_analytic in layer.parameters():
            it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])

            while not it.finished:
                idx = it.multi_index
                original_value = param[idx]

                # f(theta + eps)
                param[idx] = original_value + epsilon
                plus_loss = model._loss.forward(model._forward(X), y)

                # f(theta - eps)
                param[idx] = original_value - epsilon
                minus_loss = model._loss.forward(model._forward(X), y)

                # restore
                param[idx] = original_value

                grad_numeric = (plus_loss - minus_loss) / (4 * epsilon)
                grad_backprop = grad_analytic[idx]

                rel_error = abs(grad_numeric - grad_backprop) / max(
                    1e-8, abs(grad_numeric) + abs(grad_backprop)
                )

                if rel_error > tolerance:
                    print("FAILED")
                    print(f"Layer: {layer.__class__.__name__}")
                    print(f"Index: {idx}")
                    print(f"Numeric: {grad_numeric}")
                    print(f"Backprop: {grad_backprop}")
                    print(f"Relative error: {rel_error}")
                    return False

                it.iternext()

    print("Gradient check PASSED")
    return True


In [12]:
np.random.seed(0)

X = np.array([
    [0, 0],
    [0, 1]
], dtype=float)

y = np.array([
    [0],
    [1]
], dtype=float)

model = Model()
model.add(Dense(4))
#model.add(Tanh())
model.add(Dense(1))
#model.add(Sigmoid())


model.compile(None, loss=MSELoss())
model._forward(X)


gradient_check(model, X, y)


Running gradient check...
Gradient check PASSED


True